In [1]:
import OSLPP

In [2]:
params = OSLPP.Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, 
                  dataset='OfficeHome', source='art', target='clipart',
                  num_src_classes=25, num_total_classes=65)
OSLPP.main(params)

1089 4365
data shapes:  (1089, 512) (4365, 512)
selecting initial rejected samples at t=2
OS=56.08 OS*=55.61 unk=67.84 HOS=61.12


In [3]:
params = OSLPP.Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, 
                  dataset='OfficeHome', source='art', target='product',
                  num_src_classes=25, num_total_classes=65)
OSLPP.main(params)

1089 4439
data shapes:  (1089, 512) (4439, 512)
selecting initial rejected samples at t=2
OS=72.36 OS*=72.35 unk=72.57 HOS=72.46
